In [140]:
%run functions.ipynb

Ready: 2023-04-15 03:07:16


In [138]:
file = r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Backups\Мастер_таблица_2ГИС_Москва_230315_1745.xlsx"
cnt_graphics = 50
cnt_shars = 10
step = 1
city = 'moscow'

In [81]:
df = (pd.read_excel(file)[['Рубрика','Запросы 2022', 'Запросы 2023', 'Запросы, итого']]
      .iloc[range(1, 16, 1)]
      .set_index('Рубрика')
      .fillna(0)
      .sort_values(['Доля рынка, %', 'Рост/Падение, %'], ascending=[False,False])
     )

df['Запросы, итого'] = (df['Запросы, итого'] - min(df['Запросы, итого'])) / (max(df['Запросы, итого']) - min(df['Запросы, итого']))*300
requests_total = pd.read_excel(file)['Запросы 2023'].sum()
df['Рост/Падение, %'] = df['Запросы 2023'] / df['Запросы 2022']
df['Доля рынка, %'] = df['Запросы 2023'] / requests_total
x = (df['Доля рынка, %'])
y = (df['Рост/Падение, %'])
sizes = (df['Запросы, итого']/1000)

In [133]:
txt = ''
for cat in ['Звёзды:','Коровы:','Дети:','Собаки:']:
    txt +=  cat + ' ' + ', '.join([word[::-1].replace('/', 'и', 1)[::-1].replace('/', ',').replace(' ,', ',').lower() for word in df[df['group'] == cat].index]) + "\n"

txt = txt[::-1].replace('\n',':', 1)[::-1]

In [134]:
txt

'Звёзды: кафе, гостиницы\nКоровы: быстрое питание, торгово-развлекательные центры и моллы, супермаркеты\nДети: бары, аэропорты\nСобаки: бизнес-центры, торговые центры и универсальные магазины, кофейни, аптеки, банкоматы, парикмахерские, железнодорожные вокзалы, банки:'

In [144]:
def plot_matrix(file, city, cnt_graphics=50, cnt_shars=10, step=1):
    shar_nums = list(range(1,16))*500
    start_range = 1
    stop_range = cnt_shars*step+1
    time = str(datetime.datetime.now())[2:16].replace('-','').replace(' ','_').replace(':','')
    
    for graphic_num in tqdm(range(cnt_graphics)):
        shar_num = shar_nums.pop()

        df = pd.read_excel(file)[['Рубрика','Запросы 2022', 'Запросы 2023', 'Запросы, итого']].iloc[range(start_range, stop_range, step)].set_index('Рубрика').fillna(0)
        start_range = stop_range
        stop_range = (graphic_num+2)*cnt_shars*step+1

        df['Запросы, итого'] = (df['Запросы, итого'] - min(df['Запросы, итого'])) / (max(df['Запросы, итого']) - min(df['Запросы, итого']))*300
        requests_total = pd.read_excel(file)['Запросы 2023'].sum()
        df['Рост/Падение, %'] = df['Запросы 2023'] / df['Запросы 2022']
        df['Доля рынка, %'] = df['Запросы 2023'] / requests_total
        x = (df['Доля рынка, %'])
        y = (df['Рост/Падение, %'])
        sizes = (df['Запросы, итого']/1000)
        x_avg = (x.min()+x.max())/2
        y_avg = (y.min()+y.max())/2
        xi, yi = df['Доля рынка, %'], df['Рост/Падение, %']
        df.loc[(xi > x_avg) & (yi > y_avg), 'group'] = 'Звёзды:'
        df.loc[(xi > x_avg)  & (yi <= y_avg), 'group'] = 'Коровы:'
        df.loc[(xi <= x_avg)  & (yi > y_avg), 'group'] = 'Дети:'
        df.loc[(xi <= x_avg)  & (yi <= y_avg), 'group'] = 'Собаки:'

        plt.rcParams['figure.figsize'] = 6.89*4, 4.25*4
        plt.rcParams['figure.dpi'] = 2**8
        plt.rcParams['font.size'] = '18'
        fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\Downloads\arialnarrow.ttf")
        fig, ax = plt.subplots()
        ax.scatter(x, y, sizes, c='#C0504D', alpha=0.7)

        n = 0.02
        m = 0.02
        ax.set_ylim(ax.get_ylim()[0] * (1 - n), ax.get_ylim()[1] * (1 + n))
        ax.set_xlim(ax.get_xlim()[0] * (1 - m), ax.get_xlim()[1] * (1 + m))
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        plt.tick_params(bottom = False, left=False)

        for xi, yi, rubric, size in zip(x, y, x.index, sizes):
            rs = rubric.split(' ')
            if len(rs) > 1:
                rsl = len(rs) // 2
                rubric = ' '.join(rs[:rsl] + ['\n'] + rs[rsl:])

            shar_path = [r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Шар_{:02}.png".format(shar_num)]
            shar_img = OffsetImage(plt.imread(shar_path[0], format="png"), zoom=size)
            ab = AnnotationBbox(shar_img, (xi, yi), frameon=False)
            ax.add_artist(ab)
            plt.annotate(rubric, (xi, yi), ha='center', va='center', font=fpath, color='#404040')

        ax.hlines((y.min()+y.max())/2, x.min(), x.max(), colors='gray', linewidth=1)
        ax.vlines((x.min()+x.max())/2, y.min(), y.max(), colors='gray', linewidth=1)
        
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354_gray.png")
        newax = fig.add_axes([0.1, 0.8, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354_gray.png")
        newax = fig.add_axes([0.9, 0.1, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        
        txt = ''
        for cat in ['Звёзды:','Коровы:','Дети:','Собаки:']:
            txt +=  cat + ' ' + ', '.join([word[::-1].replace('/', 'и', 1)[::-1].replace('/', ',').replace(' ,', ',').lower() for word in df[df['group'] == cat].index]) + "\n"
        txt = txt[::-1].replace('\n',':', 1)[::-1]

        fig.savefig(f'pictures/{city}_shar_{shar_num}.png', bbox_inches='tight', transparent=True)
        add_image(f'files/{city}_shars_{time}.docx', f'pictures/{city}_shar_{shar_num}.png', word=txt)
        #plt.show()

In [145]:
file = r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Backups\Мастер_таблица_2ГИС_Москва_230315_1745.xlsx"

In [ ]:
plot_matrix(file, city, cnt_graphics=100, cnt_shars=12, step=1)

  1%|▊                                                                                 | 1/100 [00:15<24:57, 15.12s/it]

In [ ]:
plot_matrix(file, city, cnt_graphics=100, cnt_shars=12, step=1)

 17%|█████████████▊                                                                   | 17/100 [02:40<15:30, 11.20s/it]C:\Users\Mamed\Anaconda\lib\site-packages\matplotlib\axes\_base.py:2475: UserWarning: Warning: converting a masked element to nan.
  xys = np.asarray(xys)
 20%|████████████████▏                                                                | 20/100 [03:15<15:58, 11.98s/it]C:\Users\Mamed\AppData\Local\Temp\ipykernel_15512\1506920706.py:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()
 28%|██████████████████████▋                                                          | 28/100 [04:58<15:15, 12.72s/it]C:\Users\Mamed\Anaconda\lib\site-packages\matplotlib\axes\_base.py:2475: UserWarning: Warning: converting a masked element t